In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Exercise 8d.

In [2]:
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.cifar10.load_data()

def split_train_valid(data, split_number=5000):
    return data[split_number:], data[:split_number]

x_train, x_valid = split_train_valid(x_train_full)
y_train, y_valid = split_train_valid(y_train_full)

x_means = x_train.mean(axis=0)
x_stds = x_train.std(axis=0)
x_train_scaled = (x_train - x_means) / x_stds
x_valid_scaled = (x_valid - x_means) / x_stds
x_test_scaled = (x_test - x_means) / x_stds

In [3]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))

for i in range(20):
    model.add(keras.layers.Dense(100, 
                                 activation='selu', 
                                 kernel_initializer='lecun_normal'))

model.add(keras.layers.Dense(10, activation='softmax'))

In [4]:
loss = 'sparse_categorical_crossentropy'
optimizer = keras.optimizers.Nadam(learning_rate=1e-4)
n_epochs = 100
early_stopping_cb = keras.callbacks.EarlyStopping(patience=25)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('selu_cifar10_model.h5')
callbacks = [early_stopping_cb, model_checkpoint_cb]

In [5]:
model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

history = model.fit(x_train_scaled, y_train, 
                    epochs=n_epochs, 
                    validation_data=(x_valid_scaled, y_valid), 
                    callbacks=callbacks)

Epoch 1/100
1407/1407 [==============================] - 10s 6ms/step - loss: 1.8152 - acc: 0.3516 - val_loss: 1.7017 - val_acc: 0.3866
Epoch 2/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.6069 - acc: 0.4264 - val_loss: 1.5933 - val_acc: 0.4306
Epoch 3/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.5082 - acc: 0.4613 - val_loss: 1.5335 - val_acc: 0.4532
Epoch 4/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.4314 - acc: 0.4939 - val_loss: 1.4943 - val_acc: 0.4762
Epoch 5/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.3723 - acc: 0.5154 - val_loss: 1.4831 - val_acc: 0.4804
Epoch 6/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.3182 - acc: 0.5335 - val_loss: 1.4748 - val_acc: 0.4894
Epoch 7/100
1407/1407 [==============================] - 8s 6ms/step - loss: 1.2712 - acc: 0.5525 - val_loss: 1.4758 - val_acc: 0.4862
Epoch 8/100
1407/1407 [==============================]

In [6]:
model = keras.models.load_model('selu_cifar10_model.h5')
model.evaluate(x_valid_scaled, y_valid)

157/157 [==============================] - 0s 1ms/step - loss: 2.1529 - acc: 0.4814


[2.1529006958007812, 0.4814000129699707]

In [7]:
model.evaluate(x_test_scaled, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 2.1488 - acc: 0.4742


[2.148838996887207, 0.4742000102996826]